# Effect of Forcings on CAMELs Simulations

Now we can look at the output and see if there are any patterns across the variables or across basin characteristics.

First we load the imports.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import xarray as xr
import bottleneck
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE, RFECV
from sklearn.impute import SimpleImputer

<br>

### You will need to edit these paths to be your folders

In [ ]:
top = '/glade/work/ashleyvb'
folder = top+'/CAMELs'
folders = folder+'/summa_camels'

<br>

# Summary Statistics of Error on output
Let's look at some error metrics by HRU.
KGE means perfect agreement if it is 1, and <0 means the mean is a better guess. 
Bias means perfect aggreement if it is 0, and larger means larger error. 
All errors have 1's added so we don't divide by 0. 

In [ ]:
# truth data set
sim_truth = xr.open_dataset(folders+'/output/merged_day/NLDAStruth_hru.nc')
the_hru = np.array(sim_truth['hruId'])

In [ ]:
# Set forcings to hold at constant or MetSim and create dictionaries
cm_vars= ['all','airpres','airtemp','LWRadAtm','pptrate','spechum','SWRadAtm','windspd']
error_kind = ['bias','kge']
est_kind = ['constant','metsim']
seas_kind = ['YEAR','DJF','MAM','JJA','SON']
#forcing, liquid water fluxes for the soil domain, turbulent heat transfer, snow, vegetation, derived 
forc_sim = np.delete(cm_vars,0)
comp_sim=['scalarSurfaceRunoff','scalarAquiferBaseflow','scalarInfiltration','scalarRainPlusMelt','scalarSoilDrainage',
          'scalarLatHeatTotal','scalarSenHeatTotal','scalarSnowSublimation',
          'scalarSWE',
          'scalarCanopyWat',
          'scalarNetRadiation','scalarTotalET','scalarTotalRunoff','scalarTotalSoilWat']
var_sim = np.concatenate([forc_sim, comp_sim])

In [ ]:
# definitions for KGE computation
def covariance(x,y,dims=None):
    return xr.dot(x-x.mean(dims), y-y.mean(dims), dims=dims) / x.count(dims)

def correlation(x,y,dims=None):
    return covariance(x,y,dims) / (x.std(dims) * y.std(dims))

In [ ]:
# set up xarray
hrud = sim_truth['hru'] #indices here are 0 to number of basins
shape = (len(hrud), len(cm_vars),len(est_kind), len(error_kind),len(seas_kind))
dims = ('hru','var','estimation','error','season')
coords = {'hru': hrud, 'var':cm_vars, 'estimation':est_kind, 'error':error_kind, 'season':seas_kind}
error_data = xr.Dataset(coords=coords)
for s in var_sim:
    error_data[s] = xr.DataArray(data=np.full(shape, np.nan),
                                 coords=coords, dims=dims,
                                 name=s)

<br>
Now run the actual computations on KGE. This takes 35 min using all 671 basins. 

In [ ]:
%%time
truth0 = sim_truth.drop_vars('hruId').load()
for v in cm_vars:
    for c in est_kind:     
        sim0 = xr.open_dataset(folders+'/output/merged_day/NLDAS' + c + '_' + v +'_hru.nc')
        sim0 = sim0.drop_vars('hruId').load()
        for i, t in enumerate(seas_kind):     
            if i==0: 
                truth = truth0
                sim = sim0
            if i>0: 
                truth = truth0.sel(time=truth0['time.season']==t)
                sim = sim0.sel(time=sim0['time.season']==t)
                
            r = sim.mean(dim='time') #to set up xarray since xr.dot not supported on dataset and have to do loop
            for s in var_sim:         
                r[s] = correlation(sim[s],truth[s],dims='time')
            # KGE value for each hru, add 1 so no nan
            ds = 1 - np.sqrt( np.square(r-1) 
                + np.square( (sim.std(dim='time')+1)/(truth.std(dim='time')+1) - 1) 
                + np.square( (sim.mean(dim='time')+1)/(truth.mean(dim='time')+1) - 1) )
            ds0 = ds.load()
            # bias value for each hru, add 1 so no nan
            ds = np.abs(sim-truth)/(truth+1) 
            ds1 = ds.mean(dim='time').load()
            for s in var_sim:
                error_data[s].loc[:,v,c,'kge',t]  = ds0[s]
                error_data[s].loc[:,v,c,'bias',t] = ds1[s]
    print(v)

In [ ]:
# change coordinates and save incase hangs up
error_data = error_data.assign_coords(hru=sim_truth['hruId'])
error_data.to_netcdf(folder+'/regress_data/error_data.nc') 

<br>
KGE does not need to be normalized. We plot the HRU error as stack of values, with no error plotting as a height of 1 for that color. Values less than 0 are plotted as 0. 

In [ ]:
#error_data =  xr.open_dataset(folder+'/regress_data/error_data.nc') #read this incase hangs up

In [ ]:
# Setup plots
x = np.arange(len(hrud))
col_vars = ['gray','y','r','g','orange','c','m','b']
letter = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
wid = ceil(len(var_sim)/3)
inc = floor(len(hrud)/10)
if inc<1: inc=1
xtic = np.arange(0, len(hrud),inc).tolist()
xtic =[int(i) for i in xtic]
xtics =[str(i+1) for i in xtic]
labels =["V"+i for i in xtics]

<br>
Do the plotting. This takes about 8 min with all 671 basins.

In [ ]:
%%time
# Just plotting all seasons. Maybe add winter (ind 1), and summer (ind 3) if you want to see more detail. 
ind = [0]#,1,3]
seas_kind0 = [ seas_kind[i] for i in ind]
for c in est_kind:      
    for t in seas_kind0:     
        plot1 = plt.figure(1, figsize = (20,10))

        for i, s in enumerate(var_sim):
            data0 = error_data[s].loc[:,:,c,'kge',t]
            data = data0.where(data0>0,0) #make the negative values be 0
            data_Master = [0] * len(hrud)
    
            plot2 = plt.subplot(3,wid,i+1)
            for j, v in enumerate(cm_vars):
                plt.bar(height = data.loc[:,v], x = x, width = 1.0, color = col_vars[j], bottom = data_Master)
                #data_Master = [m + n for m, n in zip(data_Master, data.loc[:,v])]
                data_Master = [j+1] * len(hrud)
         
            plt.title('('+letter[i]+') '+s)
            plt.ylim(0,len(cm_vars))
            plt.xticks(xtic, labels, fontsize = 5)
            plt.yticks(np.arange(0, len(cm_vars)+.05, 1).tolist())
            plt.tick_params(axis = "x", which = "both", bottom = False, top = False)
            plt.xlabel("CAMELS basin ("+labels[0]+"-"+labels[-1]+")", fontsize = 9)
            plt.ylabel("KGE", fontsize = 9)

        plt.subplots_adjust(hspace = .4)

        for j, v in enumerate(cm_vars):
            plt.scatter([],[], color = col_vars[j], label = t + '_NLDAS_' + c + '_' + v)
        plt.figlegend(loc = 'lower right')
        plt.show()

<br>
We see that the pptrate and air pressure would be better off constant than at MetSim values (thiner orange and yellow layers in the MetSim plots), but that the air pressure does not matter in the variable calculation (except simulation of air pressure itself). Air temperature has less error in MetSim. 
By season, there is more error in the winter in both Metsim and Constant.

<br>

# Best run with 671 BASINS: Correlations of Error and Basin Attributes
We look at the basin attributes to see if there are any patterns with the error sizes. 
This code will run with fewer than 671 basins, but stronger relationships between error and basin attributes can be surmised if you look at all the basins. 
We use the Kendall non-parametric correlation based on ranks, so that error magnitude (that is likely more affected by calibrated or not calibrated parameters) is not a factor. 
The attribute file that SUMMA uses does not have many continuous variables in it, so we use the raw attribute data that would have been used to derive the SUMMA attribute file and the parameters.
TEST Budyko of each setup??

In [ ]:
# Start wih attribute data used by SUMMA. It does not have many values and a bunch of them are indices.
attrib = xr.open_dataset(folders+'/settings.v1/attributes.nc')
print(attrib)

In [ ]:
# TO LOOK AT THE 671 WITH THE SAVED STUFF
attrib = xr.open_dataset(folders+'/settings.v1/attributes.camels.v2.nc')
sim_truth = xr.open_dataset(folders+'/output/merged_dayall671/NLDAStruth_hru.nc')
hrud = sim_truth['hru'] #indices here are 0 to number of basins
the_hru = np.array(sim_truth['hruId'])
error_data =  xr.open_dataset(folder+'/regress_data/error_dataall671.nc') #read this incase hangs up
cm_vars= ['all','airpres','airtemp','LWRadAtm','pptrate','spechum','SWRadAtm','windspd']
error_kind = ['bias','kge']
est_kind = ['constant','metsim']
seas_kind = ['YEAR','DJF','MAM','JJA','SON']
forc_sim = np.delete(cm_vars,0)
comp_sim=['scalarSurfaceRunoff','scalarAquiferBaseflow','scalarInfiltration','scalarRainPlusMelt','scalarSoilDrainage',
          'scalarLatHeatTotal','scalarSenHeatTotal','scalarSnowSublimation',
          'scalarSWE',
          'scalarCanopyWat',
          'scalarNetRadiation','scalarTotalET','scalarTotalRunoff','scalarTotalSoilWat']
var_sim = np.concatenate([forc_sim, comp_sim])

In [ ]:
# Need to get all HRU ids
file_list = []
filelist = open(folders+'/settings.v1/forcingFileList.1hr.txt', 'r')
for lineNumber, line in enumerate (filelist):
   file_list.append(folders+'/forcing/1hr/'+line.strip("'\n"))
filelist.close()
extra_vars0 = xr.open_dataset(file_list[0]) 
extra_vars0 = extra_vars0.assign_coords(hru=extra_vars0['hruId'])

In [ ]:
# variables to regress, take only floats
lr_attrib = attrib.get(['hruId']) 
lr_attrib = lr_attrib.assign_coords(hru=lr_attrib['hruId'])
file_name0 = ['clim','geol','hydro','soil','topo','vege']
file_name = file_name0.copy()
n_attrib = file_name
for i, f in enumerate(file_name):
    df = pd.read_csv(folder+'/regress_data/camels_'+f+'.txt',delimiter=';')
    df['hru'] = range(0,671)
    xr_tmp = df.set_index(['hru']).to_xarray()
    xr_tmp = xr_tmp.assign_coords(hru=extra_vars0['hruId'])
    xr_tmp = xr_tmp.sel(hru=the_hru)
    xr_att = xr_tmp.drop_vars([ var for var in xr_tmp.variables if not 'float64'==xr_tmp[var].dtype ])
    xr_att = (xr_att - xr_att.mean(dim='hru', skipna=True))/xr_att.std(dim='hru', skipna=True) #normalize
    if i==0: n_attrib[i]= len(xr_att.variables)-1
    if i>0: n_attrib[i]= len(xr_att.variables)+n_attrib[i-1]
    lr_attrib =xr.merge([lr_attrib,xr_att])

In [ ]:
# Add hruId as coordinates and print results
lr_attrib = lr_attrib.drop('hruId').load()
attrib_kind = list(lr_attrib.variables.keys())
attrib_kind.remove('hru')
print(n_attrib)
print(attrib_kind)

<br>
Now run the regressions and plot. 

In [ ]:
# set up xarray, only do off KGE
attrib_num = ['clim','geol','hydro','soil','topo','vege']
shape = (len(attrib_kind), len(est_kind), len(seas_kind))
dims = ('attrib','estimation','season')
coords = {'attrib': attrib_kind, 'estimation':est_kind,'season':seas_kind}
corr_data = xr.Dataset(coords=coords)
for v in cm_vars:
    corr_data[v] = xr.DataArray(data=np.full(shape, np.nan),
                                 coords=coords, dims=dims,
                                 name=v)

In [ ]:
# for fewer basins, get rid of significance
PT = 0.01
# get number of HRUs
if len(the_hru) <25: PT = 1

In [ ]:
def r_cor(x,y, pthres = PT, direction = False):
    """
    Uses the scipy stats module to calculate a Kendall correlation test
    :pthres: Significance of the underlying test
    :direction: output only direction as output (-1 & 1)
    """
    
    # Check NA values
    nas = np.logical_or(np.isnan(x), np.isnan(y))
    if len(x[~nas]) < 10: # If fewer than 10 data return nan
        return np.nan
    if np.linalg.norm(x[~nas] - np.mean(x[~nas])) < 1e-13 * abs(np.mean(x[~nas])): #near constant attibute
         return np.nan
    if np.linalg.norm(y[~nas] - np.mean(y[~nas])) < 1e-13 * abs(np.mean(y[~nas])): #near constant error
         return np.nan
       
    
    # Run the kendalltau test
    #stat, p_value = stats.kendalltau(x[~nas], y[~nas])
    # Run the spearmanr test
    stat, p_value = stats.spearmanr(x[~nas], y[~nas])
    # Run the pearsonr test
    #stat, p_value = stats.pearsonr(x[~nas], y[~nas])
    
    # Criterium to return results in case of Significance
    if p_value < pthres:
        # Check direction
        if direction:
            if stat < 0:
                return -1
            elif stat > 0:
                return 1
        else:
            return stat
    else:
      return 0  

# The function we are going to use for applying our statistics test
def rank_correlation(x,y):
    return xr.apply_ufunc(
        r_cor, x , y
        )

# <br>
Calculate the correlations. This takes about a minute. 

In [ ]:
%%time
ds = error_data.get(comp_sim)
for a in attrib_kind:
    ds0 = lr_attrib[a]
    for c in est_kind:
        for t in seas_kind:
            for v in cm_vars:
                ds1 = ds.loc[dict(var = v,estimation=c,error = 'kge',season=t)]
                ds1 = ds1.where(ds1>-1,-1) #make the very negative values be -1
                ds1 = sum(d for d in ds1.data_vars.values())
                value = rank_correlation(ds0.values, ds1.values)
                corr_data[v].loc[a,c,t] = value

In [ ]:
# Setup plots
x = np.arange(len(attrib_kind))
col_vars = ['gray','y','r','g','orange','c','m','b']
letter = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
xtic = n_attrib
labels=file_name0

<br>
Do the plotting.

In [ ]:
%%time
# Just plotting all seasons. Maybe add winter (ind 1), and summer (ind 3) if you want to see more detail. 
ind = [0]#,1,3]
seas_kind0 = [ seas_kind[i] for i in ind]
for c in est_kind:      
    for t in seas_kind0:     
        plot1 = plt.figure(1, figsize = (7,7))
        data0 = corr_data.loc[dict(estimation=c,season=t)]
        #data = abs(data0)
        data = data0
        data_Master = [0] * len(attrib_kind)  
        for j, v in enumerate(cm_vars):
            plt.bar(height = data[v], x = x, width = 1.0, color = col_vars[j], bottom = data_Master)
            #data_Master = [m + n for m, n in zip(data_Master, data.loc[:,v])]
            data_Master = [j+1] * len(attrib_kind)
        
        plt.title('Sum Over Simulated Variables')
        plt.xticks(xtic, labels, fontsize = 6)
        #plt.ylim(0,(len(cm_vars)/1.8))
        #plt.yticks(np.arange(0, (len(cm_vars)/2.0)+.55, 0.5).tolist())
        plt.ylim(0,len(cm_vars))
        plt.yticks(np.arange(-0.5, len(cm_vars)+.05, 1).tolist())
        plt.tick_params(axis = "x", which = "both", bottom = False, top = False)
        plt.xlabel("CAMELS Attrib (a1-a52)", fontsize = 9)
        plt.ylabel("Spearman Correlation with KGE", fontsize = 9)

        for j, v in enumerate(cm_vars):
            plt.scatter([],[], color = col_vars[j], label = t + '_NLDAS_' + c + '_' + v)
        plt.figlegend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.show()

In [ ]:
# set up xarray, only do off KGE
# Just do all seasons. Maybe add winter (ind 1), and summer (ind 3) if you want to see more detail. 
ind = [0]#,1,3]
seas_kind0 = [ seas_kind[i] for i in ind]

pick_top = 5 #len(attrib_kind)
pick_coord = range(0, pick_top)

rank_coord = ['raw','rank']

shape = (len(est_kind), len(seas_kind0),pick_top, len(rank_coord))

dims = ('estimation','season','selected','rank')
coords = {'estimation':est_kind,'season':seas_kind0,'selected':pick_coord, 'rank':rank_coord}
rfe_data = xr.Dataset(coords=coords)
for v in cm_vars:
    rfe_data[v] = xr.DataArray(data=np.full(shape, np.nan),
                                 coords=coords, dims=dims,
                                 name=v)
rfe_data = rfe_data.astype(str)

<br>
This takes about a 10 seconds with all 671 subbasins. 

In [ ]:
%%time
ds = error_data.get(comp_sim)
ds =ds.fillna(0) # So don't add to total KGE
ds0 = lr_attrib
# Make the nan's the means of each attribute variable (imputer takes mean over column so need to transpose first)
imputer = SimpleImputer()
X = ds0.to_array().transpose()
X_imputed = imputer.fit_transform(X)
length = pick_top
for c in est_kind:
    for t in seas_kind0:
        ds1 = ds.loc[dict(estimation=c,error = 'kge',season=t)] # by KGE, all vars
        ds2 = ds1.rank(dim='var')/len(hrud) #by rank         
        ds1 = ds1.where(ds1>-1,-1) #make the very negative values be -1            
        for i in rank_coord: #just do rank
            if (i=='raw'): ds4 = ds1 # by actual value
            if (i=='rank'): ds4 = ds2 # by rank
            ds4 = sum(d for d in ds4.data_vars.values())
            for v in cm_vars:
                ds3 = ds4.loc[:,v]
                # Choose top some in MLE
                rfe = RFE(estimator=LinearRegression(), n_features_to_select=pick_top)
                # Automatically choose the number of features, this gives lower R2 values
                #rfe = RFECV(estimator=LinearRegression()) 

                # Do fit
                rfe.fit(X_imputed, ds3.values)
                r2 = rfe.score(X_imputed, ds3.values)
                adjusted_r2 = 1 - (1-rfe.score(X_imputed, ds3.values))*(len(ds3.values)-1)/(len(ds3.values)-X_imputed.shape[1]-1)
                print(c,t,i,v,len(np.array(attrib_kind)[rfe.support_]),r2, adjusted_r2)
                if (len(np.array(attrib_kind)[rfe.support_])<5): length = len(np.array(attrib_kind)[rfe.support_])
                rfe_data[v].loc[c,t,range(0,length),i] = np.array(attrib_kind)[rfe.support_][range(0,length)]

In [ ]:
print(rfe_data)

In [ ]:
length

In [ ]:
len(np.array(attrib_kind)[rfe.support_])